In [ ]:
import numpy as np
import scipy.stats

In [ ]:
TEAM_NUM = 2
TEAM_WEIGHTS = np.array([1,1,1,1])
TEAM_WEIGHTS = TEAM_WEIGHTS/TEAM_WEIGHTS.sum()
EXTRA_NUM = 0

RATE_CAPT_NOISE = 0.1
RATE_NOISE = 0.1

RANDOM_SEED_CAPT = False
NO_NOISE_CAPT_SEED = False
RANDOM_CAPT = True
CAPT_EQUAL = False

FLIP_MIDDLE = True

PLAYER_N = (len(TEAM_WEIGHTS) +EXTRA_NUM) * TEAM_NUM

In [ ]:
from collections import defaultdict
np.random.seed(42)

dists = defaultdict(list)
spans = defaultdict(list)
vecs = defaultdict(list)
for SNAKE in [False,True]:
    for iteration in range(10000):
        ratings = np.random.normal(size=PLAYER_N)
        capt_ratings = scipy.stats.multivariate_normal(ratings,RATE_CAPT_NOISE).rvs()
        player_ratings = scipy.stats.multivariate_normal(ratings,RATE_NOISE).rvs(TEAM_NUM)
        
        if RANDOM_CAPT:
            capts = np.arange(PLAYER_N)
            np.random.shuffle(capts)
            capts = capts[-TEAM_NUM:]
        else:
            capts = np.argsort(capt_ratings)[-TEAM_NUM:]
            #capts = np.array([_ for _ in capts[::-1]])
        
        if NO_NOISE_CAPT_SEED:
            s_idx = sorted([(ratings[i],i) for i in capts])
            capts = np.array([_[1] for _ in s_idx])
        chosen = np.zeros(PLAYER_N)
        range_s = np.arange(TEAM_NUM)
        if RANDOM_SEED_CAPT:
            np.random.shuffle(range_s)
        chosen[capts] = range_s+1
        if CAPT_EQUAL:
            ratings[capts] = 1.0

        for i in range(len(TEAM_WEIGHTS)-1):
            draft_order = list(range(TEAM_NUM))
            if (SNAKE and (i%2) == 1) or (FLIP_MIDDLE and i > 1):
                draft_order = draft_order[::-1]
            for j in draft_order:
                #print(j)
                best = max([(_,i) for i,_ in enumerate(player_ratings[j]) if chosen[i] == 0])[1]
                chosen[best] = j + 1
        rts = []
        for k in range(TEAM_NUM):
            rts.append((np.array(sorted(ratings[chosen == k+1],reverse=True)) * TEAM_WEIGHTS).sum())
        dists[SNAKE].append(np.std(rts))
        spans[SNAKE].append(np.ptp(rts))
        vecs[SNAKE].append(rts)
        #raise

In [ ]:
{k:np.mean(v) for k,v in dists.items()}

In [ ]:
{k:np.mean(v) for k,v in spans.items()}

In [ ]:
{k:np.round(np.mean(v,0),2) for k,v in vecs.items()}